In [ ]:
import serial
import struct
import torch
from PIL import Image
import io
from torchvision import transforms

PORT = "COM5"
BAUD = 115200

ser = serial.Serial(PORT, BAUD, timeout=2)

transform = transforms.Compose([
    transforms.Resize((128, 256)),
    transforms.ToTensor()
])

class WaterNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, 3, 2),
            torch.nn.ReLU(),
            torch.nn.Conv2d(16, 32, 3, 2),
            torch.nn.ReLU(),
            torch.nn.Flatten(),
            torch.nn.Linear(32 * 30 * 62, 1)
        )

    def forward(self, x):
        return self.net(x).squeeze()

model = WaterNet()
model.load_state_dict(torch.load("water_level_model.pt"))
model.eval()

def capture_image():
    ser.write(b"CAPTURE\n")
    ser.readline()  # OK
    size = struct.unpack("<I", ser.read(4))[0]
    img_bytes = ser.read(size)
    return img_bytes

img_bytes = capture_image()

img = Image.open(io.BytesIO(img_bytes)).convert("L")
x = transform(img).unsqueeze(0)

with torch.no_grad():
    level = model(x).item()

print(f"Predicted water level: {level:.2f} mm")
